# YAS CAD FDM Builder

This Jupyter notebook will convert the source tables for the Yorkshire Ambulance Service NHS Trust (YAS) computer aided despatch data into a single FDM table

In [ ]:
from FDMBuilder.FDMTable import *
from FDMBuilder.FDMDataset import *
from FDMBuilder.testing_helpers import *

DATASET_ID = "CY_FDM_YAS_999_v1"


# Leave this bit alone!
if check_dataset_exists(DATASET_ID):
    clear_dataset(DATASET_ID)
    print("Good to go!")
else:
    print("#" * 33 + " PROBLEM!! " + 33 * "#" + "\n")
    print("Something doesn't look right. Check you spelled everything correctly,\n" 
          "your dataset has been created in GCP, and you have the right permisssions\n")
    print("#" * 80)

## Copy tables and wrangle

There are currently two tables 'src_YAS_999' and 'src_YAS_CAD' that need to be combined. Duplicated rows and ResponseTypes of 'Daily Log' need to be removed.

In [ ]:
cad_1_df = FDMTable(
    source_table_id = "CY_STAGING_DATABASE.src_YAS_CAD",
    dataset_id=DATASET_ID
)

cad_2_df = FDMTable(
    source_table_id = "CY_STAGING_DATABASE.src_YAS_999",
    dataset_id=DATASET_ID
)

Copy tables to Dataset folder

In [ ]:
cad_1_df.copy_table_to_dataset()
cad_2_df.copy_table_to_dataset()

src_YAS_CAD has 42 columns, whereas src_YAS_999 has 50. We need to add these columns before the data can be joined.

In [ ]:
%%bigquery
ALTER TABLE `CY_FDM_YAS_999_v1.src_YAS_CAD`
ADD COLUMN IF NOT EXISTS DespatchCode2 STRING,
ADD COLUMN IF NOT EXISTS GeoZonePostCode STRING,
ADD COLUMN IF NOT EXISTS GovtStdTOC STRING,
ADD COLUMN IF NOT EXISTS PatientsThisResource2 STRING,
ADD COLUMN IF NOT EXISTS ResourceType2 STRING,
ADD COLUMN IF NOT EXISTS TimeCallCommenced STRING;

Create a new CAD table which contains all rows from both tables.

In [ ]:
%%bigquery
CREATE TABLE `CY_FDM_YAS_999_v1.CAD` AS (
SELECT DISTINCT * FROM
    (
        SELECT *
        FROM `CY_FDM_YAS_999_v1.src_YAS_999`
        WHERE ResourceType != 'Daily Log'
        #LIMIT 1000
    )
    UNION ALL
    (
        SELECT *
        FROM `CY_FDM_YAS_999_v1.src_YAS_CAD`
        WHERE ResourceType != 'Daily Log'
        #LIMIT 1000
    )
)

There is some variability in the presentation of dates/times in the CAD data. The WrangleDateTime function attempts to determine the type based on string length.

In [ ]:
%%bigquery
CREATE TEMP FUNCTION WrangleDateTime(x STRING)
RETURNS DATETIME
AS (
    CASE LENGTH(x)
        WHEN 19 THEN SAFE.PARSE_DATETIME("%d/%m/%Y %H:%M:%S", x)
        WHEN 16 THEN SAFE.PARSE_DATETIME("%d/%m/%Y %H:%M", x)
        WHEN 8 THEN SAFE.PARSE_DATETIME("%H:%M:%S", x)
        WHEN 5 THEN SAFE.PARSE_DATETIME("%H:%M", x)
        ELSE NULL
        END
);

CREATE TABLE `CY_FDM_YAS_999_v1.YAS_CAD` AS (

    SELECT CallNumber,
        CCG,
        GeoZonePostCode,
        TypeOfCall,
        WrangleDateTime(Date_Time_forPerf) AS Date_Time_forPerf_dt,
        # Technically, the call connect time, TimeStampT0, is the earliest call time recorded. However
        # it is not always captured. If that is the case, fallback to using the incident time, Date_Time_forPerf
        IF(WrangleDateTime(TimeStampT0) IS NULL OR EXTRACT(YEAR FROM WrangleDateTime(TimeStampT0)) = 1970, 
               WrangleDateTime(Date_Time_forPerf),
               WrangleDateTime(TimeStampT0)
          ) AS Start_dt,
        WrangleDateTime(TimeStampT0) AS TimeStampT0_dt,
        WrangleDateTime(TimeStampT1) AS TimeStampT1_dt,
        WrangleDateTime(TimeStampT2) AS TimeStampT2_dt,
        WrangleDateTime(TimeStampT3) AS TimeStampT3_dt,
        WrangleDateTime(TimeStampT4) AS TimeStampT4_dt,
        WrangleDateTime(TimeOnMDT) AS TimeOnMDT_dt,
        WrangleDateTime(TimeAllocated) AS TimeAllocated_dt,
        WrangleDateTime(TimeMobile) AS TimeMobile_dt,
        WrangleDateTime(TimeAtScene) AS TimeAtScene_dt,
        WrangleDateTime(TimeLeftScene) AS TimeLeftScene_dt,
        WrangleDateTime(TimeAtDest) AS TimeAtDest_dt,
        WrangleDateTime(TimeHandover) AS TimeHandover_dt,
        WrangleDateTime(TimeClear) AS TimeClear_dt,
        WrangleDateTime(TimeAVLMove) AS TimeAVLMove_dt,
        WrangleDateTime(PerfBestResp) AS PerfBestResp_dt,
        WrangleDateTime(PerfBestConvResp) AS PerfBestConvResp_dt,
        WrangleDateTime(PerfBestLatePatSide) AS PerfBestLatePatSide,
        HospitalAttended,
        Callsign,
        homeDPOnAlloc,
        GovtStdTOC,
        # It appears during import some variables are duplicated. On odd occassions, DespatchCode is 
        # NULL but DespatchCode2 has a value. Same for ResourceType, PatientsThisResource
        COALESCE(DespatchCode, DespatchCode2) as DispatchCode,
        COALESCE(ResourceType, ResourceType2) as Resource,
        CurrentDPOnAlloc,
        MainPatientAge,
        ChiefComplaint,
        MainPatientSex,
        methodofcall,
        AuthorisingDoctor,
        AuthorisingPractice,
        JourneyType,
        COALESCE(PatientsThisResource, PatientsThisResource2) as NumberOfPatients,
        MainPatientMobility,
        Priority,
        PosnOfResourceOnCall,
        HCPEmergency,
        HCPResponseMinutes,
        ClinicAttended,
        NHS_Number AS digest,
        PickupLocationType

    FROM `CY_FDM_YAS_999_v1.CAD`
    # LIMIT 50

)


In [ ]:
cad_df = FDMTable(
    source_table_id = "CY_FDM_YAS_999_v1.YAS_CAD",
    dataset_id=DATASET_ID
)

In [ ]:
cad_df.quick_build(
    fdm_start_date_cols="Start_dt",
    fdm_start_date_format="YMD",
    fdm_end_date_cols="TimeClear_dt",
    fdm_end_date_format="YMD"
)

Drop tables that are no longer required

In [ ]:
%%bigquery
DROP TABLE CY_FDM_YAS_999_v1.CAD;
DROP TABLE CY_FDM_YAS_999_v1.src_YAS_999;
DROP TABLE CY_FDM_YAS_999_v1.src_YAS_CAD;

In [ ]:
dataset = FDMDataset(
    dataset_id = DATASET_ID
)
dataset.build()

Add CAD_descriptor to data dictionary table

In [ ]:
%%bigquery

CREATE TABLE `CY_FDM_YAS_999_v1.CAD_dict` (
    variable_name STRING,
    CAD_descriptor STRING
)

In [ ]:
%%bigquery

INSERT `CY_FDM_YAS_999_v1.CAD_dict` (variable_name, CAD_descriptor)
VALUES('person_id', "Unique identifier for patient"),
('CallNumber', "Unique incident identifier (note can be multiple entries per incident if more than one ambulance sent to scene)"),
('CCG', "Patient's primary care Clinical Commisioning Group name"),
('GeoZonePostCode', "Outbound portion of incident postcode"),
('TypeOfCall', "Call type e.g. emergency"),
('Date_Time_forPerf_dt', "Date time of 999 call for performance ('clock start')"),
('Start_dt', "Either date time of 999 call when connected to the ambulance service from the BT operator or Date_Time_forPerf_dt if this date time is missing"),
('TimeStampT0_dt', "Time call put through to ambulance service from BT operator"),
('TimeStampT1_dt', "Time call picked up by ambulance service call handler"),
('TimeStampT2_dt', "Time address of incident verified"),
('TimeStampT3_dt', "Time problem/reason for call established"),
('TimeStampT4_dt', "Time chief complaint/symptom group established"),
('TimeOnMDT_dt', "Time incident received on ambulance mobile data terminal"),
('TimeAllocated_dt', "Time ambulance resource allocated to 999 call"),
('TimeMobile_dt', "Time ambulance mobile to incident"),
('TimeAtScene_dt', "Time ambulance arrived on scene of incident"),
('TimeLeftScene_dt', "Time ambulance left scene of incident"),
('TimeAtDest_dt', "Time ambulance arrived at destination (usually a hospital)"),
('TimeHandover_dt', "Time patient handed over by ambulance staff to hospital/destination staff"),
('TimeClear_dt', "Time ambulance clear of incident and available to respond to further emergency calls"),
('TimeAVLMove_dt', "Time from ambulance allocated to incident to ambulance detected as being mobile to incident"),
('PerfBestResp_dt', "Best response time for any resource sent to incident"),
('PerfBestConvResp_dt', "Best response time for any conveying resources"),
('PerfBestLatePatSide', "Best lateness time for any resource reaching the patient's side"),
('HospitalAttended', "Name of destination hospital"),
('Callsign', "Callsign of ambulance resource"),
('homeDPOnAlloc', "Home dispatch point of ambulance when allocated to call i.e. ambulance station resource from not current location"),
('GovtStdTOC', "999 call triage category"),
('DispatchCode', "Advanced Medical Priority Despatch Code allocated to call following call triage"),
('Resource', "Type of resource e.g. ambulance"),
('CurrentDPOnAlloc', "Current despatch point of ambulance upon allocation to call"),
('MainPatientAge', "Patient age in years"),
('ChiefComplaint', "High-level triage description (from AMPDS or 111 triage system)"),
('MainPatientSex', "Patient sex"),
('methodofcall', "Where call has originated from e.g. 999 call or inter-facility"),
('AuthorisingDoctor', "If appropriate authorising GP for 999 call"),
('AuthorisingPractice', "If appropriate authorising GP's primary care practice"),
('JourneyType', "Journey type e.g. Emergency Admission"),
('NumberOfPatients', "Number of patients on ambulance"),
('MainPatientMobility', "Mobility of patient/requirement for aids such as carry chair or stretcher"),
('Priority', "Call priority number"),
('PosnOfResourceOnCall', "Position of resource on call. If multiplel resources sent identification of order of allocation for this resource"),
('HCPEmergency', "Call identified as a healthcare professional emergency i.e. an HCP made the 999 call for a patient"),
('HCPResponseMinutes', "Response time specified by the HCP for this emergency"),
('ClinicAttended', "Desintation department or ward"),
('digest', "Pseudonymised NHS number - used as lookup for person_id"),
('PickupLocationType', "Incident location type. Pre-specified list e.g.. care home or railway station"),
('fdm_start_date', "FDM datetime for start of record. Same as Start_dt"),
('fdm_end_date', "FDM datetime for end of record. Same as TimeClear_dt")

In [ ]:
Add an additional column, CAD_descriptor to the data dictionary table...provides an explanatino of what the field actually relates to.

In [ ]:
%%bigquery

ALTER TABLE `CY_FDM_YAS_999_v1.YAS_CAD_data_dict`
ADD COLUMN CAD_descriptor STRING;

UPDATE `CY_FDM_YAS_999_v1.YAS_CAD_data_dict` A 
SET A.CAD_descriptor = B.CAD_descriptor
FROM `CY_FDM_YAS_999_v1.CAD_dict` B
WHERE A.variable_name = B.variable_name

In [ ]:
%%bigquery
DROP TABLE CY_FDM_YAS_999_v1.CAD_dict;

In [ ]:
Data quality checks

In [ ]:
%%bigquery
SELECT fdm_problem, COUNT(*) FROM `CY_FDM_YAS_999_v1.YAS_CAD_fdm_problems` GROUP BY fdm_problem

In [ ]:
%%bigquery
SELECT fdm_problem, COUNT(*) FROM `CY_FDM_YAS_999_v1.YAS_CAD_fdm_problems` WHERE GeoZonePostCode LIKE 'BD%' GROUP BY fdm_problem

In [ ]:
%%bigquery
SELECT * FROM `CY_FDM_YAS_999_v1.YAS_CAD_fdm_problems` WHERE fdm_problem = 'fdm_end_date is before fdm_start_date' LIMIT 10